# FastAI Predictions

In analogy to nnunet predictions, fastai predictions use the mean softmax probabilities of the five cross validation models constructed during training.

In [1]:
from fastai.vision import *
from fastai.callbacks.hooks import *
from fastai.utils.mem import *
from fastai.callbacks import *
from PIL import Image as PILImage
from matplotlib.cm import ScalarMappable as mpl_scalar_mappale
from matplotlib.colors import Normalize as mpl_norm

_Required functions_

In [2]:
def predict_ensemble(segmentation_item_list, paths_to_learners):
    data = segmentation_item_list
    all_names = [data.items[i].name for i in range(len(data))]
    all_preds = []
    for l in paths_to_learners:
        learner_path = Path(l)
        learn = load_learner(learner_path.parent, file=learner_path.name)
        preds = [learn.predict(d)[2].numpy() for d in data]
        all_preds.append(np.stack(preds))
    all_preds = np.stack(all_preds)
    mean_preds = np.mean(all_preds, axis=0)
    all_labels = np.argmax(mean_preds, axis=1)
    return [(all_names[i], all_labels[i, ...], mean_preds[i, ...]) for i in range(len(data))]

In [3]:
def predict_ensemble_directory(input_directory, output_directory, paths_to_learners, overlay_directory=None, num_labels=None):
    data = SegmentationItemList.from_folder(input_directory)
    predictions = predict_ensemble(data, paths_to_learners)
    for name, labels, _ in predictions:
        msk = PILImage.fromarray(labels.astype('uint8'))
        msk.save(os.path.join(output_directory, name), optimize=True)
        
        if overlay_directory:
            sm = mpl_scalar_mappale(mpl_norm(0, num_labels-1), 'viridis')
            cm_arr_rgba = sm.to_rgba(labels)
            cm_arr_rgba = (cm_arr_rgba * 255).astype('uint8')
            cm_arr_rbga_img = PILImage.fromarray(cm_arr_rgba, mode='RGBA')

            img_arr_rgb = np.array(PILImage.open(os.path.join(input_directory, name)))
            alpha_channel = np.full((512, 512, 1), 255, dtype='uint8')
            img_arr_rgba = np.concatenate((img_arr_rgb, alpha_channel), axis=-1).astype('uint8')
            img_arr_rgba_img = PILImage.fromarray(img_arr_rgba, mode='RGBA')

            ol_img = PILImage.blend(img_arr_rgba_img, cm_arr_rbga_img, 0.3)
            ol_img.save(os.path.join(overlay_directory, name), optimize=True)

# Analysis-specific predictions

These predictions are made for specific analyses conducted for the vesseg paper.

In [9]:
# set paths to exported learners 
ptl = ['/home/jacob/projects/vesseg/model/99_train_valid/99_x_val_fold_{}.pkl'.format(str(i)) for i in range(5)]

### Make predictions for interrater analysis

In [5]:
for gt_rater in ['phillip', 'alex', 'jessica']:
    
    img_dir = Path('/media/jacob/data/jacob/vesseg/comparison/'+gt_rater+'/gt/images/rc')
    out_dir = Path('/media/jacob/data/jacob/vesseg/comparison/'+gt_rater+'/pred/fastai/masks/rc')
    
    predict_ensemble_directory(img_dir, out_dir, ptl)

### Make predictions for all three test sets

In [133]:
test_sets = ['99_test', 'j_test', 'p_test']
i_dir = '/media/jacob/data/jacob/vesseg/prediction/{}/images/'
o_dir = '/media/jacob/data/jacob/vesseg/prediction/{}/preds/fastai'
ptl = ['/home/jacob/projects/vesseg/model/99_train_valid/99_x_val_fold_{}.pkl'.format(str(i)) for i in range(5)]
args = [(i_dir.format(t), o_dir.format(t), ptl) for t in test_sets]

In [136]:
for a in args: 
    predict_ensemble_directory(*a)

### Make predictions after data reconstruction

In [5]:
input_dir = '/data/jacob/vesseg/data/resized/images/'
output_dir = '/data/jacob/vesseg/data/predictions/fastai'
overlay_dir = '/data/jacob/vesseg/data/overlays/fastai'
num_labels = 3
ptl = ['/home/jacob/projects/vesseg/model/99_train_valid/99_x_val_fold_{}.pkl'.format(str(i)) for i in range(5)]
predict_ensemble_directory(input_dir, output_dir, ptl, overlay_directory=overlay_dir, num_labels=num_labels)